In [ ]:
import warnings
warnings.filterwarnings('ignore', module='jupyter_client')

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

# 1. **Import, chuẩn bị môi trường và thư viện**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# -----------------
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Dataset/online_retail_II.xlsx", sheet_name="Year 2009-2010")
print(df.shape)
df.head()

(525461, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom




| Tên biến | Ý nghĩa | Kiểu dữ liệu | Ghi chú |
|-----------|----------|---------------|---------|
| **Invoice** | Mã hóa đơn (số giao dịch). | Chuỗi / Số nguyên | Mỗi hóa đơn có thể chứa nhiều dòng hàng khác nhau. |
| **StockCode** | Mã sản phẩm (SKU). | Chuỗi | Phân biệt từng mặt hàng; có thể lặp lại ở nhiều hóa đơn. |
| **Description** | Tên sản phẩm (mô tả ngắn). | Chuỗi | Dễ có lỗi chính tả, ký tự đặc biệt; cần làm sạch. |
| **Quantity** | Số lượng sản phẩm được bán trong hóa đơn. | Số nguyên | Dương → bán ra; âm → hàng trả lại. |
| **InvoiceDate** | Thời điểm phát hành hóa đơn. | Datetime | Dùng để phân tích theo giờ, ngày, tháng, mùa,... |
| **Price** | Giá đơn vị của sản phẩm. | Float | Là giá **mỗi sản phẩm**; tổng tiền = `Quantity * Price`. |
| **Customer ID** | Mã khách hàng duy nhất. | Số / Chuỗi | Có thể thiếu (NaN) với khách chưa đăng ký. |
| **Country** | Quốc gia nơi khách hàng đặt hàng. | Chuỗi | Phân tích theo khu vực, thị trường, xu hướng địa lý. |

# 2. **EDA**

## **2.1. Thông tin tổng quan**

In [ ]:
print(df.info())
print(df.describe(include='all').T)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB
None
                 count    unique                                 top    freq  \
Invoice     525461.000 28816.000                          537434.000 675.000   
StockCode       525461      4632                              85123A    3516   
Description     522533      4681  WHITE HANGING HEART T-LIGHT H



| Đặc điểm | Nhận xét |
|-----------|-----------|
| **Số dòng** | 525,461 dòng dữ liệu → khối lượng trung bình, đủ lớn để phân tích hành vi khách hàng. |
| **Số cột** | 8 cột, tương ứng với các thông tin: Invoice, StockCode, Description, Quantity, InvoiceDate, Price, Customer ID, Country. |
| **Kiểu dữ liệu** | - `Invoice`, `StockCode`, `Description`, `Country` → **object (chuỗi)**.<br> - `Quantity` → **int64**.<br> - `Price`, `Customer ID` → **float64**.<br> - `InvoiceDate` → **datetime64[ns]** (đã được chuyển đổi đúng). |
| **Giá trị thiếu (NaN)** | - `Description`: 2,928 giá trị bị thiếu (522,533 / 525,461).<br> - `Customer ID`: bị thiếu khá nhiều (hơn 100,000 giá trị). |
| **Nhận xét tổng quát** | - Dữ liệu khá đầy đủ, tuy nhiên có **missing values** ở `Description` và `Customer ID` cần xử lý.<br> - Kiểu dữ liệu nhìn chung **đã đúng chuẩn**, đặc biệt `InvoiceDate` đã ở dạng datetime.<br> - Có thể cần tạo thêm cột như `TotalPrice = Quantity * Price` cho bước phân tích tiếp theo. |




- Có 525,461 dòng dữ liệu và 8 cột.  
- Tổng cộng 28,816 hóa đơn khác nhau.  
- Có 4,632 mã sản phẩm (StockCode) và 4,681 mô tả sản phẩm (Description).  
- Sản phẩm bán nhiều nhất là “WHITE HANGING HEART T-LIGHT HOLDER”.  
- Số lượng trung bình mỗi dòng là 10.33, có giá trị âm → thể hiện hàng trả lại.  
- Giá trung bình 4.689, có giá âm và giá rất cao → cần kiểm tra dữ liệu lỗi.  
- Thời gian giao dịch từ 01/12/2009 đến 09/12/2010.  
- Có 417,534 khách hàng có ID, phần còn lại bị thiếu.  
- Có 40 quốc gia, trong đó United Kingdom chiếm đa số.  
- Cần xử lý dữ liệu thiếu, giá trị âm và kiểm tra các giá trị bất thường trước khi phân tích.  


## **2.2. Missing values & trùng lặp**

In [ ]:
print("Missing values per column:\n", df.isna().sum())
print("Số bản ghi trùng lặp:", df.duplicated().sum())

Missing values per column:
 Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64
Số bản ghi trùng lặp: 6865


## **2.3. Thống kê cơ bản**

In [ ]:
print("Số khách hàng duy nhất:", df["Customer ID"].nunique())
print("Số sản phẩm duy nhất:", df["StockCode"].nunique())
print("Khoảng thời gian giao dịch:")
print("  Min:", df["InvoiceDate"].min())
print("  Max:", df["InvoiceDate"].max())

Số khách hàng duy nhất: 4383
Số sản phẩm duy nhất: 4632
Khoảng thời gian giao dịch:
  Min: 2009-12-01 07:45:00
  Max: 2010-12-09 20:01:00


# **3. Tiền xử lý dữ liệu**

## **3.1. Chuẩn hóa kiểu dữ liệu**

In [ ]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors='coerce')
df["Customer ID"] = df["Customer ID"].astype("Int64")
df["Quantity"] = pd.to_numeric(df["Quantity"], errors='coerce')
df["Price"] = pd.to_numeric(df["Price"], errors='coerce')

## **3.2. Loại trùng lặp và mô tả rỗng**

In [ ]:
df = df.drop_duplicates()
df["Description"] = df["Description"].astype(str).str.strip().str.lower()
df = df[df["Description"].notna()]

## **3.3. Xử lý hoàn trả / hủy đơn**

In [ ]:
df["is_return"] = df["Invoice"].astype(str).str.startswith("C") | (df["Quantity"] < 0)

df_returns = df[df["is_return"]]
df_sales = df[~df["is_return"]]

print("Số bản ghi trả/hủy:", len(df_returns))

Số bản ghi trả/hủy: 12303


## **3.4. Xử lý giá và số lượng bất thường**

In [ ]:
print(df[(df["Price"] <= 0) | (df["Quantity"] == 0)])

df["Amount"] = df["Price"] * df["Quantity"]

Q1, Q3 = df["Amount"].quantile([0.25, 0.75])
IQR = Q3 - Q1
lower_limit, upper_limit = Q1 - 1.5*IQR, Q3 + 1.5*IQR

df["Amount"] = np.clip(df["Amount"], lower_limit, upper_limit)

       Invoice StockCode   Description  Quantity         InvoiceDate  Price  \
263     489464     21733  85123a mixed       -96 2009-12-01 10:52:00  0.000   
283     489463     71477         short      -240 2009-12-01 10:52:00  0.000   
284     489467    85123A   21733 mixed      -192 2009-12-01 10:53:00  0.000   
470     489521     21646           nan       -50 2009-12-01 11:44:00  0.000   
3114    489655     20683           nan       -44 2009-12-01 17:26:00  0.000   
...        ...       ...           ...       ...                 ...    ...   
525231  538159     21324           nan       -18 2010-12-09 17:17:00  0.000   
525232  538158     20892           nan       -32 2010-12-09 17:17:00  0.000   
525233  538160     20956           nan       288 2010-12-09 17:18:00  0.000   
525234  538161    46000S  dotcom sales      -100 2010-12-09 17:25:00  0.000   
525235  538162    46000M  dotcom sales      -100 2010-12-09 17:25:00  0.000   

        Customer ID         Country  is_return  
26

## **3.5. Missing Customer ID**

In [ ]:
missing_rate = df["Customer ID"].isna().mean()
print(f"Tỷ lệ thiếu Customer ID: {missing_rate:.2%}")

if missing_rate > 0.3:
    df["Customer ID"] = df["Customer ID"].fillna(-1).astype("Int64")

Tỷ lệ thiếu Customer ID: 20.79%


## **3.6. Làm sạch StockCode**

In [ ]:
non_product_codes = ['post', 'carriage', 'amazonfee', 's', 'm']
mask = ~df["StockCode"].astype(str).str.lower().isin(non_product_codes)
df = df[mask]

# **4. Chuẩn hóa & tạo đặc trưng**

## **4.1. Đặc trưng thời gian**

In [ ]:
df["day"] = df["InvoiceDate"].dt.day
df["weekday"] = df["InvoiceDate"].dt.weekday
df["month"] = df["InvoiceDate"].dt.month
df["hour"] = df["InvoiceDate"].dt.hour
df["is_weekend"] = df["weekday"].isin([5, 6]).astype(int)

## **4.2. Đặc trưng giỏ hàng**

In [ ]:
invoice_stats = df.groupby("Invoice")["StockCode"].nunique().reset_index(name="items_per_invoice")
df = df.merge(invoice_stats, on="Invoice", how="left")

invoice_total = df.groupby("Invoice")["Amount"].sum().reset_index(name="invoice_total")
df = df.merge(invoice_total, on="Invoice", how="left")

## **4.3. RFM features (per user)**

In [ ]:
rfm = (
    df.groupby("Customer ID")
    .agg({
        "InvoiceDate": "max",
        "Invoice": "nunique",
        "Amount": "sum"
    })
    .rename(columns={
        "InvoiceDate": "last_purchase",
        "Invoice": "Frequency",
        "Amount": "Monetary"
    })
)

latest_date = df["InvoiceDate"].max()
rfm["Recency"] = (latest_date - rfm["last_purchase"]).dt.days
rfm = rfm[["Recency", "Frequency", "Monetary"]].reset_index()

rfm.head()

,Customer ID,Recency,Frequency,Monetary
0,12346,66,14,343.810
1,12347,2,2,1273.595
2,12348,73,1,221.160
3,12349,42,3,1941.715
4,12351,10,1,300.930


## **4.4. User-product interaction features**

In [ ]:
user_item_count = df.groupby(["Customer ID", "StockCode"]).size().reset_index(name="times_bought")
last_buy = df.groupby(["Customer ID", "StockCode"])["InvoiceDate"].max().reset_index(name="last_purchase_date")

df = df.merge(user_item_count, on=["Customer ID", "StockCode"], how="left")
df = df.merge(last_buy, on=["Customer ID", "StockCode"], how="left")

## **4.5. Làm sạch mô tả sản phẩm**

In [ ]:
df["Description_clean"] = df["Description"].apply(
    lambda x: re.sub(r"[^a-zA-Z0-9\s]", "", x).strip().lower()
)

# **5. Huấn luyện**

## **5.1. Chuẩn bị dữ liệu ở mức hóa đơn**

In [ ]:
sales = df[~df["is_return"]].copy()

basket = sales.groupby(["Invoice", "Description"])["Quantity"].sum().unstack().fillna(0)
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

print("Shape của basket:", basket.shape)
basket.head()

Shape của basket: (21814, 4516)


Description  *boombox ipod classic  *usb office glitter lamp  \
Invoice                                                        
489434                           0                         0   
489435                           0                         0   
489436                           0                         0   
489437                           0                         0   
489438                           0                         0   

Description  *usb office mirror ball  10 colour spaceboy pen  \
Invoice                                                        
489434                             0                       0   
489435                             0                       0   
489436                             0                       0   
489437                             0                       0   
489438                             0                       0   

Description  11 pc ceramic tea set polkadot  \
Invoice                                       
489434                                    0   
489435                                    0   
489436                                    0   
489437                                    0   
489438                                    0   

Description  12 ass zinc christmas decorations  12 coloured party balloons  \
Invoice                                                                      
489434                                       0                           0   
489435                                       0                           0   
489436                                       0                           0   
489437                                       0                           0   
489438                                       0                           0   

Description  12 daisy pegs in wood box  12 egg house painted wood  \
Invoice                                                             
489434                               0                          0   
489435                               0                          0   
489436                               0                          0   
489437                               0                          0   
489438                               0                          0   

Description  12 ivory rose peg place settings  \
Invoice                                         
489434                                      0   
489435                                      0   
489436                                      0   
489437                                      0   
489438                                      0   

Description  12 message cards with envelopes  12 mini toadstool pegs  \
Invoice                                                                
489434                                     0                       0   
489435                                     0                       0   
489436                                     0                       0   
489437                                     0                       0   
489438                                     0                       0   

Description  12 pencil small tube woodland  12 pencils small tube posy  \
Invoice                                                                  
489434                                   0                           0   
489435                                   0                           0   
489436                                   0                           0   
489437                                   0                           0   
489438                                   0                           0   

Description  12 pencils small tube red retrospot  \
Invoice                                            
489434                                         0   
489435                                         0   
489436                                         0   
489437                                         0   
489438                                         0   

Descript


Chuẩn bị dữ liệu cho phân tích giỏ hàng (Market Basket Analysis).

- Loại bỏ các giao dịch trả hàng (`is_return = True`).  
- Gom nhóm dữ liệu theo hóa đơn và sản phẩm.  
- Chuyển dữ liệu từ dạng “số lượng bán” sang **ma trận nhị phân (0/1)**:  
  - 1 → sản phẩm có trong hóa đơn.  
  - 0 → sản phẩm không có.  
- Kết quả cuối là **basket matrix** – đầu vào cho thuật toán **Apriori** hoặc **FP-Growth** để khai thác **luật kết hợp giữa các sản phẩm**.  

## **5.2. FP-Growth**

In [ ]:
basket_bool = basket.astype(bool)

from mlxtend.frequent_patterns import fpgrowth, association_rules

frequent_itemsets = fpgrowth(basket_bool, min_support=0.01, use_colnames=True)
frequent_itemsets.sort_values("support", ascending=False).head(10)

,support,itemsets
48,0.152,(white hanging heart t-light holder)
516,0.093,(regency cakestand 3 tier)
0,0.075,(strawberry ceramic trinket box)
6,0.065,(assorted colour bird ornament)
87,0.065,(pack of 72 retro spot cake cases)
83,0.061,(60 teatime fairy cake cases)
7,0.061,(home building block word)
584,0.057,(jumbo bag red retrospot)
62,0.056,(lunch bag red spotty)
131,0.054,(jumbo storage bag suki)


Áp dụng thuật toán **FP-Growth** để tìm ra **các tập mặt hàng xuất hiện thường xuyên** (frequent itemsets) trong giỏ hàng của khách hàng.  

- Tìm các tổ hợp sản phẩm có tần suất xuất hiện cùng nhau đạt ngưỡng hỗ trợ tối thiểu `min_support=0.01` (tức là xuất hiện trong ít nhất 1% giao dịch).  

- Bảng cho thấy các mặt hàng được mua thường xuyên nhất trong toàn bộ dữ liệu.
- Giá trị support thể hiện tần suất xuất hiện của từng mặt hàng trong các hóa đơn.

## **5.3. Sinh luật kết hợp**

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules = rules.sort_values("lift", ascending=False)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
778,(poppy's playhouse bedroom),(poppy's playhouse livingroom),0.015,0.012,0.010,0.704,58.873,1.000,0.010,3.343,0.998,0.631,0.701,0.781
779,(poppy's playhouse livingroom),(poppy's playhouse bedroom),0.012,0.015,0.010,0.858,58.873,1.000,0.010,6.951,0.995,0.631,0.856,0.781
777,(poppy's playhouse livingroom),(poppy's playhouse kitchen),0.012,0.016,0.011,0.897,57.185,1.000,0.011,9.515,0.994,0.634,0.895,0.790
776,(poppy's playhouse kitchen),(poppy's playhouse livingroom),0.016,0.012,0.011,0.684,57.185,1.000,0.011,3.129,0.998,0.634,0.680,0.790
775,(poppy's playhouse kitchen),(poppy's playhouse bedroom),0.016,0.015,0.013,0.801,54.958,1.000,0.012,4.956,0.997,0.710,0.798,0.831
774,(poppy's playhouse bedroom),(poppy's playhouse kitchen),0.015,0.016,0.013,0.862,54.958,1.000,0.012,7.114,0.996,0.710,0.859,0.831
750,(green regency teacup and saucer),(pink regency teacup and saucer),0.017,0.012,0.010,0.609,51.119,1.000,0.010,2.529,0.997,0.553,0.605,0.733
751,(pink regency teacup and saucer),(green regency teacup and saucer),0.012,0.017,0.010,0.858,51.119,1.000,0.010,6.909,0.992,0.553,0.855,0.733
382,(set/10 blue spotty party candles),(set/10 pink spotty party candles),0.015,0.018,0.012,0.794,44.777,1.000,0.011,4.777,0.992,0.563,0.791,0.727
383,(set/10 pink spotty party candles),(set/10 blue spotty party candles),0.018,0.015,0.012,0.659,44.777,1.000,0.011,2.889,0.995,0.563,0.654,0.727


## **5.4. Lọc luật theo business requirement**

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules = rules.sort_values("lift", ascending=False)

rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
778,(poppy's playhouse bedroom),(poppy's playhouse livingroom),0.015,0.012,0.010,0.704,58.873,1.000,0.010,3.343,0.998,0.631,0.701,0.781
779,(poppy's playhouse livingroom),(poppy's playhouse bedroom),0.012,0.015,0.010,0.858,58.873,1.000,0.010,6.951,0.995,0.631,0.856,0.781
777,(poppy's playhouse livingroom),(poppy's playhouse kitchen),0.012,0.016,0.011,0.897,57.185,1.000,0.011,9.515,0.994,0.634,0.895,0.790
776,(poppy's playhouse kitchen),(poppy's playhouse livingroom),0.016,0.012,0.011,0.684,57.185,1.000,0.011,3.129,0.998,0.634,0.680,0.790
775,(poppy's playhouse kitchen),(poppy's playhouse bedroom),0.016,0.015,0.013,0.801,54.958,1.000,0.012,4.956,0.997,0.710,0.798,0.831
774,(poppy's playhouse bedroom),(poppy's playhouse kitchen),0.015,0.016,0.013,0.862,54.958,1.000,0.012,7.114,0.996,0.710,0.859,0.831
750,(green regency teacup and saucer),(pink regency teacup and saucer),0.017,0.012,0.010,0.609,51.119,1.000,0.010,2.529,0.997,0.553,0.605,0.733
751,(pink regency teacup and saucer),(green regency teacup and saucer),0.012,0.017,0.010,0.858,51.119,1.000,0.010,6.909,0.992,0.553,0.855,0.733
382,(set/10 blue spotty party candles),(set/10 pink spotty party candles),0.015,0.018,0.012,0.794,44.777,1.000,0.011,4.777,0.992,0.563,0.791,0.727
383,(set/10 pink spotty party candles),(set/10 blue spotty party candles),0.018,0.015,0.012,0.659,44.777,1.000,0.011,2.889,0.995,0.563,0.654,0.727


In [ ]:
# Lọc luật có support ≥ 0.02 (2%)
rules_over_2pct = rules[rules['support'] >= 0.02]

# Sắp xếp theo lift giảm dần để thấy các cặp mạnh nhất
rules_over_2pct = rules_over_2pct.sort_values('lift', ascending=False)

rules_over_2pct.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
291,(vintage snap cards),(vintage heads and tails card game),0.046,0.034,0.020,0.444,12.970,1.000,0.019,1.736,0.967,0.341,0.424,0.519
290,(vintage heads and tails card game),(vintage snap cards),0.034,0.046,0.020,0.595,12.970,1.000,0.019,2.357,0.956,0.341,0.576,0.519
436,(wooden frame antique white),(wooden picture frame white finish),0.052,0.043,0.028,0.531,12.225,1.000,0.025,2.038,0.969,0.408,0.509,0.585
437,(wooden picture frame white finish),(wooden frame antique white),0.043,0.052,0.028,0.639,12.225,1.000,0.025,2.624,0.960,0.408,0.619,0.585
40,(chocolate hot water bottle),(hot water bottle tea and sympathy),0.043,0.047,0.022,0.516,11.082,1.000,0.020,1.971,0.950,0.327,0.493,0.494
41,(hot water bottle tea and sympathy),(chocolate hot water bottle),0.047,0.043,0.022,0.471,11.082,1.000,0.020,1.812,0.954,0.327,0.448,0.494
721,(heart of wicker small),(heart of wicker large),0.047,0.050,0.025,0.535,10.658,1.000,0.023,2.045,0.951,0.350,0.511,0.519
720,(heart of wicker large),(heart of wicker small),0.050,0.047,0.025,0.503,10.658,1.000,0.023,1.916,0.954,0.350,0.478,0.519
254,(sweetheart ceramic trinket box),(strawberry ceramic trinket box),0.046,0.075,0.036,0.795,10.577,1.000,0.033,4.516,0.949,0.431,0.779,0.640
255,(strawberry ceramic trinket box),(sweetheart ceramic trinket box),0.075,0.046,0.036,0.485,10.577,1.000,0.033,1.854,0.979,0.431,0.461,0.640
